In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 384, 286

train_data_dir = 'dataset_small/train'
validation_data_dir = 'dataset_small/val'

num_classes = 5
# number of training examples (including all the classes)
nb_train_samples = 10*5 #3042*5
nb_validation_samples = 15*5
epochs = 1
batch_size = 10


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1. / 255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Using TensorFlow backend.


Found 50 images belonging to 5 classes.
Found 75 images belonging to 5 classes.
Epoch 1/1
5/5 [==============================] - 10s - loss: 5.3661 - acc: 0.2000 - val_loss: 1.6211 - val_acc: 0.0143


In [2]:
model.save_weights('2_classification_dataset.h5')

In [3]:
# done saving the model

In [4]:
# exporting the model to .pb

In [5]:
K.set_learning_phase(0)
config = model.get_config()
weights = model.get_weights()


In [6]:
from keras.models import model_from_config
# new_model = model_from_config(config)
new_model = Sequential.from_config(config)
new_model.set_weights(weights)

new_model.save('2_classification_new_dataset_test_model.h5')

In [7]:
# model export starts here

from keras.models import load_model
import tensorflow as tf
import os
import os.path as osp
from keras import backend as K

weight_file_path = './2_classification_new_dataset_test_model.h5'
num_output = 1
prefix_output_node_names_of_final_network = 'output_node'
export_path =  './export_2_classification' # where to save the exported graph
output_graph_name = 'constant_graph_weights_new_dataset.pb'

K.set_learning_phase(0)
pred = [None]*num_output
pred_node_names = [None]*num_output
net_model = load_model(weight_file_path)
for i in range(num_output):
    pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
    pred[i] = tf.identity(net_model.output, name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)



sess = K.get_session()

if 1:
    f = 'only_the_graph_def.pb.ascii'
    tf.train.write_graph(sess.graph.as_graph_def(), export_path, f, as_text=True)
    print('saved the graph definition in ascii format at: ', osp.join(export_path, f))




from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, export_path, output_graph_name, as_text=False)
print('saved the constant graph (ready for inference) at: ', osp.join(export_path, output_graph_name))



for n in constant_graph.node:
    print(n.name)




/Users/siddkuma/anaconda/lib/python2.7/site-packages/keras/models.py:245: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


('output nodes names are: ', ['output_node0'])
('saved the graph definition in ascii format at: ', './export_2_classification/only_the_graph_def.pb.ascii')
INFO:tensorflow:Froze 10 variables.
Converted 10 variables to const ops.
('saved the constant graph (ready for inference) at: ', './export_2_classification/constant_graph_weights_new_dataset.pb')
conv2d_1_input_2
conv2d_1_2/kernel
conv2d_1_2/kernel/read
conv2d_1_2/bias
conv2d_1_2/bias/read
conv2d_1_2/convolution
conv2d_1_2/BiasAdd
activation_1_2/Relu
max_pooling2d_1_2/MaxPool
conv2d_2_2/kernel
conv2d_2_2/kernel/read
conv2d_2_2/bias
conv2d_2_2/bias/read
conv2d_2_2/convolution
conv2d_2_2/BiasAdd
activation_2_2/Relu
max_pooling2d_2_2/MaxPool
conv2d_3_2/kernel
conv2d_3_2/kernel/read
conv2d_3_2/bias
conv2d_3_2/bias/read
conv2d_3_2/convolution
conv2d_3_2/BiasAdd
activation_3_2/Relu
max_pooling2d_3_2/MaxPool
flatten_1_2/Shape
flatten_1_2/strided_slice/stack
flatten_1_2/strided_slice/stack_1
flatten_1_2/strided_slice/stack_2
flatten_1_2/str

In [8]:
# testing and inference starts here

In [9]:
import tensorflow as tf
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
import numpy as np

def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
    return graph

model = load_graph('./export_2_classification/constant_graph_weights_new_dataset.pb')

In [10]:
for n in model.as_graph_def().node:
    print n.name

prefix/conv2d_1_input_2
prefix/conv2d_1_2/kernel
prefix/conv2d_1_2/kernel/read
prefix/conv2d_1_2/bias
prefix/conv2d_1_2/bias/read
prefix/conv2d_1_2/convolution
prefix/conv2d_1_2/BiasAdd
prefix/activation_1_2/Relu
prefix/max_pooling2d_1_2/MaxPool
prefix/conv2d_2_2/kernel
prefix/conv2d_2_2/kernel/read
prefix/conv2d_2_2/bias
prefix/conv2d_2_2/bias/read
prefix/conv2d_2_2/convolution
prefix/conv2d_2_2/BiasAdd
prefix/activation_2_2/Relu
prefix/max_pooling2d_2_2/MaxPool
prefix/conv2d_3_2/kernel
prefix/conv2d_3_2/kernel/read
prefix/conv2d_3_2/bias
prefix/conv2d_3_2/bias/read
prefix/conv2d_3_2/convolution
prefix/conv2d_3_2/BiasAdd
prefix/activation_3_2/Relu
prefix/max_pooling2d_3_2/MaxPool
prefix/flatten_1_2/Shape
prefix/flatten_1_2/strided_slice/stack
prefix/flatten_1_2/strided_slice/stack_1
prefix/flatten_1_2/strided_slice/stack_2
prefix/flatten_1_2/strided_slice
prefix/flatten_1_2/Const
prefix/flatten_1_2/Prod
prefix/flatten_1_2/stack/0
prefix/flatten_1_2/stack
prefix/flatten_1_2/Reshape
pre

In [12]:
# img = plt.imread('./dataset/val/WW/139.jpg')
# img = np.reshape(img,[1,256,256,3])
# img = img.astype(float)
# img = img/255.0
input_node = model.get_tensor_by_name("prefix/conv2d_1_input_2:0")
output_node = model.get_tensor_by_name("prefix/output_node0:0")



In [77]:
with tf.Session(graph=model) as sess:
        # Note: we didn't initialize/restore anything, everything is stored in the graph_def
        y_out = sess.run(output_node, feed_dict={
            input_node: img
        })
        print(y_out)

[[ 1.]]


In [78]:
import numpy as np
test_data_dir = 'dataset/val'

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='binary',
    shuffle=False)


Found 60 images belonging to 2 classes.


In [79]:
preds = new_model.predict_generator(generator=test_generator, steps=60)

In [80]:
pred_cl = preds > 0.5
pred_cl = pred_cl.astype(np.int)
pred_for_file = zip(test_generator.filenames, pred_cl, preds)

for i in pred_for_file:
    print(i)

('WW/121.jpg', array([0]), array([ 0.00123855], dtype=float32))
('WW/122.jpg', array([0]), array([  6.20405553e-05], dtype=float32))
('WW/123.jpg', array([0]), array([ 0.2508404], dtype=float32))
('WW/124.jpg', array([0]), array([ 0.00026049], dtype=float32))
('WW/125.jpg', array([0]), array([  2.69901157e-05], dtype=float32))
('WW/126.jpg', array([0]), array([ 0.00014258], dtype=float32))
('WW/127.jpg', array([0]), array([ 0.0006878], dtype=float32))
('WW/128.jpg', array([0]), array([ 0.0001845], dtype=float32))
('WW/129.jpg', array([0]), array([  1.49825373e-05], dtype=float32))
('WW/130.jpg', array([0]), array([  6.02863875e-05], dtype=float32))
('WW/131.jpg', array([0]), array([ 0.00248698], dtype=float32))
('WW/132.jpg', array([0]), array([ 0.00056203], dtype=float32))
('WW/133.jpg', array([0]), array([  6.17742480e-05], dtype=float32))
('WW/134.jpg', array([0]), array([ 0.00069535], dtype=float32))
('WW/135.jpg', array([0]), array([ 0.00046818], dtype=float32))
('WW/136.jpg', arr

In [81]:
preds.shape

(60, 1)